<a href="https://colab.research.google.com/github/Existanze54/sirius-machine-learning-2025/blob/main/Seminars/BioInf/S7_Cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Основные методы ML

### Семинар 7: Кластеризация

#### Снижение размерности и кластеризация. Классификация химических веществ

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
sns.set_style("whitegrid")

### Задача 1: Кластеризация и снижение размерности

Загрузим датасет с прошлого семинара:

In [ ]:
!wget https://data.bioml.ru/htdocs/courses/bioml/classic_ml/unsupervised/dim_reduction/data/gse53625.tar.gz -O gse53625.tar.gz
!tar xvzf gse53625.tar.gz

In [ ]:
escc = pd.read_csv("./data/gse53625_expression.csv", index_col=0)
escc_meta = pd.read_csv("./data/gse53625_metadata.csv", index_col=0)

In [ ]:
X = escc.values.T
y = escc_meta["Sample type"]
batch = escc_meta["Dataset"]

Осуществите **стандартизацию данных**.

In [ ]:
# your code here

Произведите **обучение модели** t-SNE и **визуализацию**.

_Указание_: Задайте параметры t-SNE `perplexity=3.0`, `random_state=42`.

In [ ]:
# your code here

**Обучите модель K-Means** на данных сниженной размерности.

In [ ]:
# your code here

Посчитайте **индекс Рэнда** для полученного разбиения.

In [ ]:
from sklearn.metrics import adjusted_rand_score
# your code here

Теперь **обучите K-Means на исходных данных** и отобразите полученные метки на графике t-SNE.

In [ ]:
# your code here

Посчитайте индекс Рэнда еще раз.

In [ ]:
# your code here

**Ответьте на вопросы**:

* Когда надо осуществлять кластеризацию: до снижения размерности или после?
* Подходит ли для этой задачи метод DBSCAN? Объясните свой ответ.

---

### Задача 2: Кластеризация химических веществ.

In [ ]:
!pip install rdkit

In [ ]:
!pip install deepchem

In [ ]:
!pip install umap-learn

#### Молекулярные фингерпринты

Идея в том, что надо закодировать молекулярную структуру в численную последовательность. Как правило, в массив ноликов и единичек длиной 1000-4000 бит. Каждый бит фингерпринта соответствует фрагменту молекулы. Существуют разные алгоритмы хэширования фрагментов, но это не меняет сути.

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from IPython.display import display, Image
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import SimilarityMaps, rdMolDraw2D
from IPython.display import SVG

In [ ]:
ibu = Chem.MolFromSmiles('CC(C)CC1=CC=C(C=C1)C(C)C(=O)O')
AllChem.Compute2DCoords(ibu)

In [ ]:
display(ibu)

In [ ]:
bit_info = {}
fp = AllChem.GetMorganFingerprintAsBitVect(ibu, radius=2, bitInfo=bit_info)
bit_info

**Интерпретация того, что выше, выглядит так:**

* бит 1 встретился дважды:
 * у атома 1 в радиусе 0;
 * у атома 10 в радиусе 0.
* бит 79 встретился однажды:
 * у атома 3 в радиусе 1.
* и т.д.

In [ ]:
display(Draw.DrawMorganBit(ibu, 1, bit_info, useSVG=True))
display(Draw.DrawMorganBit(ibu, 310, bit_info, useSVG=True))
display(Draw.DrawMorganBit(ibu, 389, bit_info, useSVG=True))
display(Draw.DrawMorganBit(ibu, 857, bit_info, useSVG=True))
display(Draw.DrawMorganBit(ibu, 900, bit_info, useSVG=True))

При помощи фингерпринтов можно сравнивать молекулы, исходя из *предположения*, что у похожих молекул похожие фингерпринты.

In [ ]:
mol2 = Chem.MolFromSmiles('CC(C)CC1=CC=C(C=C1)C(C1=CN=NN1)(C)C(=O)O')
AllChem.Compute2DCoords(mol2)

In [ ]:
display(mol2)

In [ ]:
drawer = rdMolDraw2D.MolDraw2DSVG(400, 400)
similarity = SimilarityMaps.GetSimilarityMapForFingerprint(ibu, mol2, SimilarityMaps.GetMorganFingerprint, draw2d=drawer)

drawer.FinishDrawing()
svg = drawer.GetDrawingText()
display(SVG(svg))

Красненькое - непохожее, зелёное - похожее.

**Загрузим датасет.** Clinfox с данными по лекарствам, прошедшим клинические испытания и не прошедшим вследствие токсичности

In [ ]:
import deepchem as dc
import numpy as np


tasks, datasets, transformers = dc.molnet.load_clintox(featurizer='ECFP', splitter="random")
train_dataset, valid_dataset, test_dataset = datasets

In [ ]:
!pip install datasets
from datasets import load_dataset
ds = load_dataset("zpn/clintox")
df = ds["train"].to_pandas()
df = df[df["smiles"].apply(lambda x: Chem.MolFromSmiles(x) is not None)]
df = df.reset_index(drop=True)

In [ ]:
import numpy as np
from rdkit.Chem import AllChem

def featurize_ecfp(smiles, radius=2, n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    return np.array(fp)

X = np.vstack(df["smiles"].apply(featurize_ecfp))
y = df["target"].values

In [ ]:
from deepchem.splits import RandomSplitter
from deepchem.data import NumpyDataset

dc_dataset = NumpyDataset(X, y)
splitter = RandomSplitter()

train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(
    dc_dataset,
    frac_train=0.8,
    frac_valid=0.1,
    frac_test=0.1,
    seed=123
)


In [ ]:
train_dataset.X

In [ ]:
train_dataset.y.reshape(1, -1)[0]

In [ ]:
train_X = np.concatenate([train_dataset.X, valid_dataset.X])
train_Y = np.concatenate([train_dataset.y, valid_dataset.y])

In [ ]:
train_X.shape, train_Y.shape

In [ ]:
train_Y

In [ ]:
int(sum(train_Y)), len(train_Y)

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
from umap import UMAP

sns.set_style("whitegrid")

In [ ]:
umap = UMAP(random_state=3)
X = umap.fit_transform(train_X)

In [ ]:
plt.figure(figsize=(8, 6))
plt.title('UMAP plot', size=24)
plt.xlabel('coordinate 1', size=16)
plt.ylabel('coordinate 2', size=16)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=train_Y);

**Скластеризуйте молекулы при помощи метода K-means, отрисуйте UMAP-проекцию с покраской по метке кластера.**

In [ ]:
from sklearn.cluster import KMeans
# your code here

**Сделайте вывод, всегда ли кластеризация по схожести фингерпринтов коррелирует с биомедицинскими характеристиками вещества.**

### Задача 3. Филогенетическая реконструкция.

<img src="https://kodomo.fbb.msu.ru/FBB/year_20/ml/clustering/phylorec.png" alt="Drawing" width= "700px;"/>

Среди всех методов реконструкции филогении можно выделить две группы: символьно-ориентированные методы и дистанционные методы.
Первые принимают на вход выравнивание последовательностей, вторые - матрицу попарных расстояний между последовательностями.

In [ ]:
!wget https://kodomo.fbb.msu.ru/FBB/year_20/ml/clustering/alignfam.fasta

In [ ]:
!pip install biopython

In [ ]:
from Bio import Phylo, AlignIO
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor

In [ ]:
align = AlignIO.read('alignfam.fasta', 'fasta')

In [ ]:
print(align)

In [ ]:
calculator = DistanceCalculator('identity')
distMatrix = calculator.get_distance(align)
print(distMatrix)

In [ ]:
constructor = DistanceTreeConstructor()
UPGMA_tree = constructor.upgma(distMatrix)

In [ ]:
UPGMA_tree.ladderize()
Phylo.draw_ascii(UPGMA_tree)

**Повторите то же самое с подсчётом матрицы расстояний с моделью 'blosum62'.**

In [ ]:
# your code here